# Creatring BIDS dataset

## Creating a dataset description

In [ ]:
addpath(genpath(fullfile(pwd, '..', 'src')));

cfg.dir.output = fullfile(pwd, 'output');

cfg.task.name = 'test task';

cfg.subject.subjectNb = 1;
cfg.subject.runNb = 1;

cfg.testingDevice = 'mri';

cfg.eyetracker.do = 1;

cfg.verbose = false;

In [ ]:
cfg.bids.datasetDescription.Name = 'my new bids data set';
cfg.bids.datasetDescription.BIDSVersion =  '1.3';
cfg.bids.datasetDescription.License = 'CC-BY';
cfg.bids.datasetDescription.Authors = {'John Wick', 'Napoléon Bonaparte'};
cfg.bids.datasetDescription.Acknowledgements = '';
cfg.bids.datasetDescription.HowToAcknowledge = '';
cfg.bids.datasetDescription.Funding = {''};
cfg.bids.datasetDescription.ReferencesAndLinks = {''};
cfg.bids.datasetDescription.DatasetDOI = '';

[cfg] = createFilename(cfg);

In [ ]:
mkdir(fullfile(cfg.dir.outputSubject, cfg.fileName.modality));

createDatasetDescription(cfg)

In [ ]:
!tree

In [ ]:
!cat output/source/dataset_description.json

## fMRI

### fMRI specific name suffixes

You can specify some extra information for you fMRI runs as some of this information can be important for filenaming or to add to some JSON files.

In [ ]:
cfg.mri.repetitionTime = 2.1;
cfg.mri.contrastEnhancement = [];
cfg.mri.phaseEncodingDirection = [];
cfg.mri.reconstruction = 'fast';
cfg.mri.echo = '1';
cfg.mri.acquisition = 'slow';

In [ ]:
[cfg] = createFilename(cfg);

In [ ]:
cfg.fileName.events

### Saving a `_bold.json` file

There is bare minimum functionality to create a `_bold.json` file for fMRI runs: the information in there wil be that required to pass a BIDS validation but no more. You should REALLY get most of the information for `bold.json` from your favorite DICOM to Nifti converter

This function is there because it can help put in JSON format some of the information that would NOT be available to a converter (usually information about your task).

In [ ]:
createBoldJson(cfg)

In [ ]:
!tree

In [ ]:
!cat output/source/sub-001/ses-001/func/sub-001*bold_date-*.json

### Adding extra info to your `_bold.json`

One thing this function CAN do though is help you save some of the information about your experiment in a json format rather than a `.mat` format. The former being a simple text file you can easily open on any computer.

For example if you have defined some other parameters in another structure.

In [ ]:
%% Timing info

% Time between blocs in secs
parameters.timing.IBI = 1.8 * 3; % 8;
% Time between events in secs
parameters.timing.ISI = 0.1;
% Number of seconds before the motion stimuli are presented
parameters.timing.onsetDelay = .1;
% Number of seconds after the end all the stimuli before ending the run
parameters.timing.endDelay = .1;

parameters.timing.eventDuration = 0.8; % second

%% Visual Stimulation

% Speed in visual angles / second
parameters.dot.speed = 30;
% Coherence Level (0-1)
parameters.dot.coherence = 1;
% Number of dots per visual angle square.
parameters.dot.density = .1;
% Dot life time in seconds
parameters.dot.lifeTime = 10;
% proportion of dots killed per frame
parameters.dot.proportionKilledPerFrame = 0;
% Dot Size (dot width) in visual angles.
parameters.dot.size = 1;

In [ ]:
createBoldJson(cfg, parameters)

In [ ]:
!cat output/source/sub-001/ses-001/func/sub-001*bold_date-*.json

## Converting your data set from source to raw

Before we do that, let's adding some actual data to the dataset.

In [ ]:

logFile.extraColumns = {'target_position', 'target_type'};

logFile = saveEventsFile('open', cfg, logFile);

logFile(1,1).onset = 2;
logFile(1,1).trial_type = 'motion_up';
logFile(1,1).duration = 1;
logFile(1,1).target_position = 'left';
logFile(1,1).target_type = 'audio';

logFile(2,1).onset = 6;
logFile(2,1).trial_type = 'static';
logFile(2,1).duration = 1.2;
logFile(2,1).target_position = 'right';
logFile(2,1).target_type = 'visual';

logFile = saveEventsFile('save', cfg, logFile);

saveEventsFile('close', cfg, logFile);

In [ ]:
!tree

The very basic conversion here mostly entails:
- adding empty README and CHANGES files in the `source` folder
- copying the content of the `source` folder into a `raw` folder
- removing the date suffixes
- compressing `_stim.tsv` into `_stim.tsv.gz`

In [ ]:
convertSourceToRaw(cfg)

In [ ]:
!tree